In [1]:
from src.requirements import *

In [ ]:
def collate_padding(batch):
    return rnn_utils.pad_sequence(batch, batch_first=True, padding_value=0)

In [ ]:
data = [torch.tensor([1,2]), torch.tensor([3,4,5,6,7,8,9]), torch.tensor([6])]
dataloader = DataLoader(data, batch_size=3, collate_fn=collate_padding)

In [ ]:
for batch in dataloader:
    print(batch)

In [4]:
x = torch.randn(16,10)
x

tensor([[ 0.2038, -0.3411,  1.3240,  2.4554,  0.9712, -0.7575,  0.3378, -0.6811,
          1.6843, -1.2972],
        [ 1.1451,  1.4667,  0.4219, -1.4867,  1.9243,  0.5414,  0.7109, -0.3683,
          0.2644, -1.5856],
        [-0.4536, -0.9458, -0.9486,  0.2513,  0.8617, -0.2937,  0.2850, -1.6226,
          0.0208,  2.2829],
        [ 1.1768,  0.6962,  0.7399,  0.1068, -0.5827,  0.1715, -0.0469,  0.7773,
         -0.1537,  0.2645],
        [-0.0663,  0.4869,  0.2739, -1.4928, -2.2394,  0.0294,  0.7866,  0.0166,
         -2.1113, -1.1183],
        [-0.5473,  0.2525,  1.8703, -1.6365, -1.0391, -0.9974,  2.3577,  0.7151,
          0.0063, -1.3196],
        [ 0.5124, -0.9498,  0.3696, -1.6469,  0.3281,  1.1025,  0.5407, -1.0411,
          2.0084,  0.7621],
        [ 0.9851, -0.6033,  0.4891,  1.5071, -0.9564,  0.9629,  0.0259,  0.7140,
         -2.1237,  0.6357],
        [-0.1728, -0.3488,  2.3962,  0.9643, -0.5325, -0.9775, -0.9039, -0.9646,
         -0.6997, -0.4547],
        [-0.6285,  

In [3]:
x.shape

torch.Size([3, 2])